In [1]:
import os
import re
import json

In [2]:
parent_dir = "rej_sampling_outputs"
output_dir = "processed_data"
filenames = [
    "bestmove_rejsampling_4000_correct_20250527-055643.json",
    "legalmoves_rejsampling_4000_correct_20250527-055643.json",
    "predictmove_rejsampling_4000_correct_20250527-055643.json",
    "worstmove_rejsampling_4000_correct_20250527-055643.json",
]

# Helper processing function
HEADER_PATTERN = re.compile(
    r"<\|header_start\|>(\w+)<\|header_end\|>\n?(.*?)(?=(<\|header_start\|>|<\|eot\|>|$))",
    re.DOTALL
)

def process_sample(sample):
    result = []
    prompt = sample['prompt']
    completion = sample['completion']

    for match in HEADER_PATTERN.finditer(prompt):
        role, content = match.group(1), match.group(2).strip()
        content = re.sub(r"<\|(?!header_start\|)(?!header_end\|)[^>]*\|>", "", content)
        if role == "system":
            pass
        elif role == "user":
            pass
        elif role == "assistant":
            continue        
        else:
            raise ValueError(f"Improper role to be processed -- role: {role}.")
        if content:
            result.append((role, content))

    # Also process assistant data
    completion = re.sub(r'(<\|eot\|>|<eot_id>)\s*$', '', completion).strip()
    if completion:
        result.append(('assistant', completion))
    return result

outputs = []
for file in filenames:
    with open(os.path.join(parent_dir, file), "r", encoding="utf-8") as f:
        data = json.load(f)
        print(f"{file}: {len(data)}")
        for sample in data:
            outputs.append({"chat": process_sample(sample)})

# Save down
print(f"Final length of processed data: {len(outputs)}")
with open(f"{output_dir}/rejsampling_clean_{len(outputs)}.jsonl", "w", encoding="utf-8") as f:
    for item in outputs:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

bestmove_rejsampling_4000_correct_20250527-055643.json: 1164
legalmoves_rejsampling_4000_correct_20250527-055643.json: 352
predictmove_rejsampling_4000_correct_20250527-055643.json: 723
worstmove_rejsampling_4000_correct_20250527-055643.json: 1030
Final length of processed data: 3269
